In [2]:
import numpy as np
import cv2

In [3]:
# Read the video input
vid = cv2.VideoCapture('./test.avi')

# Get the frame count
n_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

# Width and height of each frame
w = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

# FPS
fps = vid.get(cv2.CAP_PROP_FPS)

# Set up output
# fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# Output format
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('test_out.mp4', fourcc, fps, (w, h))

In [4]:
# Read first frame
_, prev = vid.read()

# Convert to Grayscale
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

In [5]:
# Initialize the transformation matrix
transforms = np.zeros((n_frames - 1, 3), np.float32)

for i in range(n_frames - 1):
    # Feature points of the previous frame
    # The feature points are obtained using the Shi-Tomasi corner detection algorithm
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=30, blockSize=3)

    # Read the next frame, if it doesn't exist, exit the loop
    flag, frame = vid.read()
    if not flag:
        break

    # Convert to Grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate the Optical flow using the Lucas-Kanade method
    frame_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prev_pts, None)

    # Sanity check
    assert frame_pts.shape == prev_pts.shape

    # Only store the valid pts
    idx = np.where(status == 1)[0]
    prev_pts = prev_pts[idx]
    frame_pts = frame_pts[idx]

    # Obtain the transformation matrix
    mat = cv2.estimateAffine2D(prev_pts, frame_pts)[0]

    # Translation components
    dx = mat[0][2]
    dy = mat[1][2]

    # Rotation component
    dtheta = np.arctan2(mat[1, 0], mat[0, 0])

    # Store the components corresponding to each frame
    transforms[i] = [dx ,dy, dtheta]

    # Store the current frame
    prev_gray = frame_gray

In [6]:
# Obtain the trajectory as a cumulative sum of the dx, dy and dtheta components
trajectory = np.cumsum(transforms, axis=0)

In [7]:
# Moving average filter
def mav_filter(curve, radius):
    window_size = 2*radius + 1
    filter = np.ones(window_size) / window_size
    # Padding
    curve_padded = np.lib.pad(curve, (radius, radius), 'edge')
    curve_smooth = np.convolve(curve_padded, filter, mode='same')
    # Remove padding
    curve_output = curve_smooth[radius:-radius]

    return curve_output

In [8]:
# Smoothen the trajectory
def smooth(trajectory, radius=7):
    trajectory_smooth = np.copy(trajectory)

    for i in range(3):
        trajectory_smooth[:, i] = mav_filter(trajectory[:, i], radius)

    return trajectory_smooth

In [9]:
# Obtain the smooth trajectory
trajectory_smooth = smooth(trajectory)

# Compute the difference in trajectories
trajectory_difference = trajectory_smooth - trajectory

# Obtain the new transformation components
transforms_smooth = transforms + trajectory_difference

In [10]:
# Fix the border by scaling the image about its center
def fix_border(frame):
    n = frame.shape
    T = cv2.getRotationMatrix2D((n[1]/2, n[0]/2), 0, 1.04)
    frame_out = cv2.warpAffine(frame, T, (n[1], n[0]))

    return frame_out

In [11]:
# Reset stream to first frame
vid.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Transform the frame using the new transformation components to obtain the stabilized frame
for i in range(n_frames - 1):
    flag, frame = vid.read()
    if not flag:
        break

    # Transformation components for the current matrix
    dx = transforms_smooth[i, 0]
    dy = transforms_smooth[i, 1]
    dtheta = transforms_smooth[i, 2]
    
    # Construct the transformation matrix
    mat = np.zeros((2, 3), np.float32)
    mat[0, 0] = np.cos(dtheta)
    mat[0, 1] = -np.sin(dtheta)
    mat[1, 0] = np.sin(dtheta)
    mat[1, 1] = np.cos(dtheta)
    mat[0, 2] = dx
    mat[1, 2] = dy

    # Apply the transformation
    frame_stabilized = cv2.warpAffine(frame, mat, (w, h))
    frame_stabilized = fix_border(frame_stabilized)

    # Write to file
    frame_out = cv2.hconcat([frame, frame_stabilized])

    # If too big, resize
    if(frame_out.shape[1] > 1920):
        frame_out = cv2.resize(frame_out, (w, h))
    
    cv2.imshow('frame', frame_out)
    cv2.waitKey(10)
    out.write(frame_out)

vid.release()
out.release()
cv2.destroyAllWindows()